# News Collection
Pull today's headline from the NYT API

In [1]:
! pip install requests

In [ ]:
import requests

# Replace with your actual NYT API key
API_KEY = 'yp31hTcsHNjRBY98C8l6OsleFf8tRAOr'
URL = f'https://api.nytimes.com/svc/topstories/v2/home.json?api-key={API_KEY}'

response = requests.get(URL)

title = ""
abstract = ""

if response.status_code == 200:
    data = response.json()
    if data['results']:
        # Assume the first article is the main front-page article
        main_article = data['results'][0]
        title = main_article['title']
        abstract = main_article['abstract']
        
        print(f"Title: {title}\n")
        print(f"Abstract: {abstract}\n")
    else:
        print("No articles found.")
else:
    print(f"Error: {response.status_code}, {response.text}")


Title: Francis’ Death Silences a Voice for the Voiceless

Abstract: As democratic values and alliances were being turned upside down, the pope was a consistent moral guidepost. Who can play that role now?



# NLP Analysis
Calculate sentiment of news articles

In [11]:
! pip install nltk

In [14]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

stop_words = set(stopwords.words("english"))
tokenized = word_tokenize(title + " " + abstract)
text = [word.lower() for word in tokenized if word.isalnum() and word.lower() not in stop_words]

sentiment_analyzer = SentimentIntensityAnalyzer()

sentiment= sentiment_analyzer.polarity_scores(' '.join(tokenized))

print(sentiment)

{'neg': 0.111, 'neu': 0.743, 'pos': 0.146, 'compound': 0.0516}


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/maggiehollis/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Mood Mapping
Convert sentiment scores to audio features

In [22]:
# def sentiment_to_audio_features(compound):
#     if compound <= -0.5:
#         valence = 0.1
#         energy = 0.2
#         tempo = 70
#         acousticness = 0.9
#     elif compound <= 0:
#         valence = 0.3
#         energy = 0.4
#         tempo = 90
#         acousticness = 0.7
#     elif compound <= 0.5:
#         valence = 0.6
#         energy = 0.6
#         tempo = 110
#         acousticness = 0.4
#     else:
#         valence = 0.9
#         energy = 0.8
#         tempo = 130
#         acousticness = 0.2
#     return {'valence': valence, 'energy': energy, 'tempo': tempo, 'acousticness': acousticness}

# features = sentiment_to_audio_features(sentiment['compound'])
# print(features)
def sentiment_to_mood(compound):
    if compound <= -0.5:
        return "sad"
    elif compound <= 0.0:
        return "calm"
    elif compound <= 0.5:
        return "happy"
    else:
        return "energetic"

mood = sentiment_to_mood(sentiment['compound'])
print(f"Mood: {mood}")



Mood: happy


# Song Recs
Search for songs in the spotify API that have audio features we matched with the mood

In [23]:
import requests

GENIUS_API_TOKEN = 'CATpWX4OIUMhY4HYjWl8QjUBJJTwzRLWKxLS88W50qawiEy2pvtzE5nPaPsXrdPh'
headers = {'Authorization': f'Bearer {GENIUS_API_TOKEN}'}

search_url = 'https://api.genius.com/search'
params = {'q': f'{mood} song'}  # e.g., 'happy song'

response = requests.get(search_url, headers=headers, params=params)
data = response.json()

for hit in data['response']['hits'][:5]:
    song = hit['result']
    print(f"{song['full_title']} - {song['url']}")


Happy Song by Bring Me The Horizon - https://genius.com/Bring-me-the-horizon-happy-song-lyrics
Весёлая Песня (Happy Song) by Егор Крид (Egor Kreed) & MORGENSHTERN - https://genius.com/Egor-kreed-and-morgenshtern-happy-song-lyrics
The Happy Song by Imogen Heap - https://genius.com/Imogen-heap-the-happy-song-lyrics
Счастливая песня (Happy Song) by Markul - https://genius.com/Markul-happy-song-lyrics
Life's a Happy Song by Jason Segel (Ft. Amy Adams, Feist, Mickey Rooney & Walter (Muppet)) - https://genius.com/Jason-segel-lifes-a-happy-song-lyrics
